# Análise sobre Contratos
---

### 1. Colunas Adicionais

1. Adicionar coluna que identifica se o contrato foi expirado (data_fim < data_atual)
2. Adicionar coluna de período de contrato

#### 1.1 Tempo médio de contrato por categoria
Objetivo: entender a duração média dos contratos com base na sua categoria (mensal, anual, trimestral, semestral e bianual)

In [1]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt


# Conectar ao banco de dados SQLite
conn = sqlite3.connect(r'C:\Users\Usuario\OneDrive\Documentos\final-project\final-project\seguros.db')

df_dict = pd.read_csv(r'C:\Users\Usuario\OneDrive\Documentos\final-project\final-project\data\P18_dicionario_dados.csv', encoding='utf-8', sep=',')
df_cancel = pd.read_csv(r'C:\Users\Usuario\OneDrive\Documentos\final-project\final-project\data\P18_cancelamentos.csv', encoding='utf-8', sep=',')
df_contratos = pd.read_csv(r'C:\Users\Usuario\OneDrive\Documentos\final-project\final-project\data\P18_contratos.csv', encoding='utf-8', sep=',')
df_clientes= pd.read_csv(r'C:\Users\Usuario\OneDrive\Documentos\final-project\final-project\data\P18_clientes.csv', encoding='utf-8', sep=',')
df_clientes.to_sql('clientes', conn, if_exists='replace', index=False)
df_contratos.to_sql('contratos', conn, if_exists='replace', index=False)
df_cancel.to_sql('cancelamentos', conn, if_exists='replace', index=False)

# Carregar as tabelas como DataFrames do pandas
df_clientes = pd.read_sql_query("SELECT * FROM clientes", conn)
df_contratos = pd.read_sql_query("SELECT * FROM contratos", conn)
df_cancel = pd.read_sql_query("SELECT * FROM cancelamentos", conn)

1.1.1 Entendendo quantos contratos temos nas três modalidades de duração de contrato.


In [4]:
query_contrato_agroup_categ = """
WITH contratos_duracao AS (
  SELECT 
    id_contrato,
    DATE(data_inicio) AS inicio,
    DATE(data_fim) AS fim,
    julianday(data_fim) - julianday(data_inicio) AS duracao_dias,
    CASE 
      WHEN julianday(data_fim) - julianday(data_inicio) <= 31 THEN 'Mensal'
      WHEN julianday(data_fim) - julianday(data_inicio) BETWEEN 32 AND 92 THEN 'Trimestral'
      WHEN julianday(data_fim) - julianday(data_inicio) BETWEEN 93 AND 183 THEN 'Semestral'
      WHEN julianday(data_fim) - julianday(data_inicio) BETWEEN 184 AND 366 THEN 'Anual'
      WHEN julianday(data_fim) - julianday(data_inicio) > 366 THEN 'Bianual'
      ELSE 'Indefinido'
    END AS tipo_duracao
  FROM contratos
)

SELECT 
  tipo_duracao,
  COUNT(*) AS total_contratos,
  ROUND(AVG(duracao_dias), 2) AS media_dias
FROM contratos_duracao
GROUP BY tipo_duracao
ORDER BY media_dias;

"""
df_contrato_agroup_categ = pd.read_sql_query(query_contrato_agroup_categ, conn)
df_contrato_agroup_categ

,tipo_duracao,total_contratos,media_dias
0,Semestral,4006,180.0
1,Anual,4017,365.0
2,Bianual,3977,730.0


1.1.2 Entendendo quantos contratos e a média de contratos cancelados nos seguintes períodos: mensal, trimestral, semestral, anual e bianual.


In [8]:
query_contrato_cancelado_agroup_categ = """
WITH contratos_cancelados AS (
  SELECT 
    c.id_contrato,
    DATE(c.data_inicio) AS inicio,
    DATE(ca.data_cancelamento) AS cancelamento,
    julianday(ca.data_cancelamento) - julianday(c.data_inicio) AS duracao_dias,
    CASE 
      WHEN julianday(ca.data_cancelamento) - julianday(c.data_inicio) <= 31 THEN 'Mensal'
      WHEN julianday(ca.data_cancelamento) - julianday(c.data_inicio) BETWEEN 32 AND 92 THEN 'Trimestral'
      WHEN julianday(ca.data_cancelamento) - julianday(c.data_inicio) BETWEEN 93 AND 183 THEN 'Semestral'
      WHEN julianday(ca.data_cancelamento) - julianday(c.data_inicio) BETWEEN 184 AND 366 THEN 'Anual'
      WHEN julianday(ca.data_cancelamento) - julianday(c.data_inicio) > 366 THEN 'Bianual'
      ELSE 'Indefinido'
    END AS tipo_duracao
  FROM contratos c
  JOIN cancelamentos ca ON c.id_contrato = ca.id_contrato
)

SELECT 
  tipo_duracao,
  COUNT(*) AS total_cancelados,
  ROUND(AVG(duracao_dias), 2) AS media_dias_cancelamento
FROM contratos_cancelados
GROUP BY tipo_duracao
ORDER BY media_dias_cancelamento;

"""
df_contrato_cancelado_agroup_categ = pd.read_sql_query(query_contrato_cancelado_agroup_categ, conn)
df_contrato_cancelado_agroup_categ

,tipo_duracao,total_cancelados,media_dias_cancelamento
0,Mensal,12,30.42
1,Trimestral,213,60.76
2,Semestral,1027,166.30
3,Anual,842,316.96
4,Bianual,426,550.06


1.1.3 Entendendo a duração média dos contratos até o cancelamento pelos tipos de durações originais do contrato, sendo: semestral, anual e bianual.

In [12]:
query_contrato_agroup_categ_cancel = """ 
WITH contratos_cancelados AS (
  SELECT 
    c.id_contrato,
    DATE(c.data_inicio) AS inicio,
    DATE(c.data_fim) AS fim,
    DATE(ca.data_cancelamento) AS cancelamento,
    
    -- Duração até o cancelamento
    julianday(ca.data_cancelamento) - julianday(c.data_inicio) AS duracao_ate_cancelamento,
    
    -- Duração total do contrato original
    julianday(c.data_fim) - julianday(c.data_inicio) AS duracao_total_contrato,
    
    -- Classificação baseada na duração total original
    CASE 
      WHEN julianday(c.data_fim) - julianday(c.data_inicio) <= 31 THEN 'Mensal'
      WHEN julianday(c.data_fim) - julianday(c.data_inicio) BETWEEN 32 AND 92 THEN 'Trimestral'
      WHEN julianday(c.data_fim) - julianday(c.data_inicio) BETWEEN 93 AND 183 THEN 'Semestral'
      WHEN julianday(c.data_fim) - julianday(c.data_inicio) BETWEEN 184 AND 366 THEN 'Anual'
      WHEN julianday(c.data_fim) - julianday(c.data_inicio) > 366 THEN 'Bianual'
      ELSE 'Indefinido'
    END AS tipo_duracao_original
  FROM contratos c
  JOIN cancelamentos ca ON c.id_contrato = ca.id_contrato
)

SELECT 
  tipo_duracao_original,
  COUNT(*) AS total_cancelados,
  ROUND(AVG(duracao_ate_cancelamento), 2) AS media_dias_ate_cancelar,
  ROUND(AVG(duracao_total_contrato), 2) AS media_dias_contrato_original
FROM contratos_cancelados
GROUP BY tipo_duracao_original
ORDER BY tipo_duracao_original;
"""
df__contrato_agroup_categ_cancel = pd.read_sql_query(query_contrato_agroup_categ_cancel, conn)
df__contrato_agroup_categ_cancel

,tipo_duracao_original,total_cancelados,media_dias_ate_cancelar,media_dias_contrato_original
0,Anual,826,280.58,365.0
1,Bianual,802,383.64,730.0
2,Semestral,892,163.52,180.0


1.3 Houve troca de plano antes do cancelamento?


In [18]:
query_troca_plano_antes_cancel = """ 
SELECT 
  c2.tipo_seguro AS novo_tipo_seguro,
  COUNT(*) AS total_contratos_apos_cancelamento
FROM contratos c1
JOIN cancelamentos ca 
  ON c1.id_contrato = ca.id_contrato
JOIN contratos c2 
  ON c1.id_cliente = c2.id_cliente
  AND DATE(c2.data_inicio) > DATE(ca.data_cancelamento)
GROUP BY c2.tipo_seguro
ORDER BY total_contratos_apos_cancelamento DESC;
"""
df_troca_plano_antes_cancel = pd.read_sql_query(query_troca_plano_antes_cancel, conn)
df_troca_plano_antes_cancel

,novo_tipo_seguro,total_contratos_apos_cancelamento
0,Vida,400
1,Residencial,366
2,Empresarial,346
3,Auto,342
4,Saúde,324


1.3.1 Entendendo o total de clientes que trocaram de plano após cancelamentos


In [24]:
query_total_contratos_pos_cancel = """ 
SELECT 
  COUNT(DISTINCT c1.id_cliente) AS total_trocas_apos_cancelamento
FROM contratos c1
JOIN cancelamentos ca ON c1.id_contrato = ca.id_contrato
JOIN contratos c2 
  ON c1.id_cliente = c2.id_cliente
  AND DATE(c2.data_inicio) > DATE(ca.data_cancelamento);
"""
df_total_contratos_pos_cancel = pd.read_sql(query_total_contratos_pos_cancel, conn)
df_total_contratos_pos_cancel

,total_trocas_apos_cancelamento
0,919


1.3.2 Entendendo por qual tipo de plano após cancelado, foi trocado.

In [20]:
query_plano_ant_pos = """
SELECT 
  c1.tipo_seguro AS tipo_seguro_cancelado,
  c2.tipo_seguro AS novo_tipo_seguro,
  COUNT(*) AS total_trocas
FROM contratos c1
JOIN cancelamentos ca 
  ON c1.id_contrato = ca.id_contrato
JOIN contratos c2 
  ON c1.id_cliente = c2.id_cliente
  AND c1.id_contrato != c2.id_contrato
WHERE DATE(c2.data_inicio) > DATE(ca.data_cancelamento)
GROUP BY tipo_seguro_cancelado, novo_tipo_seguro
ORDER BY total_trocas DESC;
"""
df_plano_ant_pos = pd.read_sql_query(query_plano_ant_pos, conn)
df_plano_ant_pos

,tipo_seguro_cancelado,novo_tipo_seguro,total_trocas
0,Residencial,Vida,97
1,Residencial,Empresarial,92
2,Empresarial,Saúde,84
3,Empresarial,Residencial,83
4,Empresarial,Vida,83
5,Auto,Vida,82
6,Auto,Residencial,81
7,Residencial,Auto,80
8,Saúde,Auto,78
9,Vida,Vida,75


In [38]:
Os que trocaram o plano pelo mesmo plano

SyntaxError: invalid syntax (646637097.py, line 1)

1.3.4 Entendendo quantos clientes cancelaram pra ter um novo contrato e quantos cancelaram para finalizar relacionamento.

In [26]:
query_clientes_renovados_perdidos = """ 
WITH cancelamentos_clientes AS (
  SELECT 
    c1.id_cliente,
    ca.data_cancelamento
  FROM contratos c1
  JOIN cancelamentos ca ON c1.id_contrato = ca.id_contrato
),

clientes_com_novo_contrato AS (
  SELECT DISTINCT c1.id_cliente
  FROM cancelamentos_clientes c1
  JOIN contratos c2 
    ON c1.id_cliente = c2.id_cliente
   AND DATE(c2.data_inicio) > DATE(c1.data_cancelamento)
)

-- Contar quantos voltaram e quantos não voltaram
SELECT 
  CASE 
    WHEN c1.id_cliente IN (SELECT id_cliente FROM clientes_com_novo_contrato) THEN 'Cancelou e voltou'
    ELSE 'Cancelou e não voltou'
  END AS status_cliente,
  COUNT(DISTINCT c1.id_cliente) AS total_clientes
FROM cancelamentos_clientes c1
GROUP BY status_cliente;
"""
df_clientes_renovados_perdidos = pd.read_sql_query(query_clientes_renovados_perdidos, conn)
df_clientes_renovados_perdidos

,status_cliente,total_clientes
0,Cancelou e não voltou,1054
1,Cancelou e voltou,919


1.3.5 Vendo a porcentagem dos clientes que renovaram e quantos cancelaram


In [28]:
query_renovou_volta_mais = """ 
-- 1. Identificar clientes com múltiplos contratos
WITH contratos_ordenados AS (
  SELECT 
    id_cliente,
    id_contrato,
    DATE(data_inicio) AS data_inicio,
    DATE(data_fim) AS data_fim
  FROM contratos
),

clientes_com_renovacao AS (
  SELECT DISTINCT c1.id_cliente
  FROM contratos_ordenados c1
  JOIN contratos_ordenados c2 
    ON c1.id_cliente = c2.id_cliente
   AND c1.id_contrato != c2.id_contrato
   AND DATE(c2.data_inicio) > DATE(c1.data_fim)  -- Renovação detectada
),

clientes_cancelaram AS (
  SELECT DISTINCT c.id_cliente
  FROM contratos c
  JOIN cancelamentos ca ON c.id_contrato = ca.id_contrato
)

-- 2. Comparar taxa de cancelamento
SELECT 
  CASE 
    WHEN r.id_cliente IS NOT NULL THEN 'Renovou ao menos 1 vez'
    ELSE 'Nunca renovou'
  END AS grupo_cliente,
  
  COUNT(DISTINCT c.id_cliente) AS total_clientes,
  COUNT(DISTINCT can.id_cliente) AS total_cancelaram,
  
  ROUND(
    CAST(COUNT(DISTINCT can.id_cliente) AS FLOAT) / 
    COUNT(DISTINCT c.id_cliente) * 100, 2
  ) AS taxa_cancelamento_percentual

FROM contratos c
LEFT JOIN clientes_com_renovacao r ON c.id_cliente = r.id_cliente
LEFT JOIN clientes_cancelaram can ON c.id_cliente = can.id_cliente
GROUP BY grupo_cliente;

"""

df_renovou_volta_mais = pd.read_sql_query(query_renovou_volta_mais, conn)
df_renovou_volta_mais

,grupo_cliente,total_clientes,total_cancelaram,taxa_cancelamento_percentual
0,Nunca renovou,2246,723,32.19
1,Renovou ao menos 1 vez,2303,1250,54.28


1.3.6 Clientes que não renovaram por tipo de contrato

In [30]:
query_contrato_nao_renovou = """ 
WITH contratos_finalizados AS (
  SELECT 
    id_cliente,
    id_contrato,
    tipo_seguro,
    DATE(data_fim) AS data_fim
  FROM contratos
  WHERE DATE(data_fim) < DATE('now')
),

sem_renovacao AS (
  SELECT cf.*
  FROM contratos_finalizados cf
  LEFT JOIN contratos c2 
    ON cf.id_cliente = c2.id_cliente
    AND DATE(c2.data_inicio) > DATE(cf.data_fim)
  WHERE c2.id_contrato IS NULL  -- ou seja, não renovou
)

SELECT 
  tipo_seguro,
  COUNT(DISTINCT id_cliente) AS clientes_nao_renovaram
FROM sem_renovacao
GROUP BY tipo_seguro
ORDER BY clientes_nao_renovaram DESC;
"""
df_contrato_nao_renovou = pd.read_sql_query(query_contrato_nao_renovou, conn)
df_contrato_nao_renovou


,tipo_seguro,clientes_nao_renovaram
0,Auto,725
1,Vida,698
2,Empresarial,685
3,Residencial,677
4,Saúde,656


1.4 Motivo de cancelamento mais frequente

In [31]:
query_motivo_cancel_agroup = """ 
SELECT 
  motivo_cancelamento,
  COUNT(*) AS total_cancelamentos
FROM cancelamentos
GROUP BY motivo_cancelamento
ORDER BY total_cancelamentos DESC;
"""
df_motivo_calcel_agroup = pd.read_sql_query(query_motivo_cancel_agroup, conn)
df_motivo_calcel_agroup

,motivo_cancelamento,total_cancelamentos
0,Atendimento ruim,520
1,Mudança de seguradora,519
2,Preço alto,506
3,Problemas no sinistro,488
4,Cobertura insuficiente,487


1.4.1 Agrupamento por avaliação da experiência

Verificar se algum motivo de cancelamento está associado a avaliações mais positivas ou negativas.

In [33]:
query_motivo_cancel_av_exp_agroup = """ 
SELECT 
  motivo_cancelamento,
  avaliacao_experiencia_cancelamento,
  COUNT(*) AS total
FROM cancelamentos
GROUP BY motivo_cancelamento, avaliacao_experiencia_cancelamento
ORDER BY motivo_cancelamento, total DESC;
"""

df_motivo_cancel_av_exp_agroup = pd.read_sql_query(query_motivo_cancel_av_exp_agroup, conn)
df_motivo_cancel_av_exp_agroup

,motivo_cancelamento,avaliacao_experiencia_cancelamento,total
0,Atendimento ruim,Boa,187
1,Atendimento ruim,Neutra,171
2,Atendimento ruim,Ruim,162
3,Cobertura insuficiente,Boa,166
4,Cobertura insuficiente,Neutra,164
5,Cobertura insuficiente,Ruim,157
6,Mudança de seguradora,Boa,192
7,Mudança de seguradora,Ruim,167
8,Mudança de seguradora,Neutra,160
9,Preço alto,Ruim,170


1.5 Cancelamentos por canal

In [36]:
query_cancel_por_canal = """ 
SELECT 
  canal_cancelamento,
  COUNT(*) AS total
FROM cancelamentos
GROUP BY canal_cancelamento
ORDER BY total DESC;
"""
df_cancel_por_canal = pd.read_sql_query(query_cancel_por_canal, conn)
df_cancel_por_canal


,canal_cancelamento,total
0,App,663
1,Telefone,647
2,Site,611
3,Agência,599


1.5.1 Existe uma relação entre canal de cancelamento e avaliação da experiencia de cancelamento?

In [34]:
query_canal_versus_aval = """ 
SELECT 
  canal_cancelamento,
  avaliacao_experiencia_cancelamento,
  COUNT(*) AS total
FROM cancelamentos
GROUP BY canal_cancelamento, avaliacao_experiencia_cancelamento
ORDER BY canal_cancelamento, total DESC;
"""
df_canal_versus_aval = pd.read_sql_query(query_canal_versus_aval, conn)
df_canal_versus_aval

,canal_cancelamento,avaliacao_experiencia_cancelamento,total
0,Agência,Boa,206
1,Agência,Neutra,200
2,Agência,Ruim,193
3,App,Boa,235
4,App,Ruim,230
5,App,Neutra,198
6,Site,Boa,220
7,Site,Neutra,196
8,Site,Ruim,195
9,Telefone,Neutra,228


1.6 Satisfação da última avaliação por tipo de contrato


In [37]:
query_satisf_ultima_avl_agroup_contrato = """ 
SELECT 
  tipo_seguro,
  satisfacao_ultima_avaliacao,
  COUNT(*) AS total
FROM contratos
GROUP BY tipo_seguro, satisfacao_ultima_avaliacao
ORDER BY tipo_seguro, total DESC;
"""
df_satisf_ultima_avl_agroup_contrato = pd.read_sql_query(query_satisf_ultima_avl_agroup_contrato, conn)
df_satisf_ultima_avl_agroup_contrato

,tipo_seguro,satisfacao_ultima_avaliacao,total
0,Auto,Média,862
1,Auto,Baixa,808
2,Auto,Alta,759
3,Empresarial,Baixa,855
4,Empresarial,Alta,790
5,Empresarial,Média,778
6,Residencial,Média,820
7,Residencial,Alta,775
8,Residencial,Baixa,757
9,Saúde,Média,803


Total de contratos ativos

In [39]:
query_total_contratos_ativos = """
SELECT COUNT(*) AS total_contratos_ativos
FROM contratos c
LEFT JOIN cancelamentos ca ON c.id_contrato = ca.id_contrato
WHERE ca.id_contrato IS NULL  -- contrato não cancelado
  AND DATE('now') BETWEEN DATE(c.data_inicio) AND DATE(c.data_fim)
;

"""
df_total_contratos_ativos = pd.read_sql_query(query_total_contratos_ativos, conn)
df_total_contratos_ativos

,total_contratos_ativos
0,3660
